In [0]:
class SilverLayer:
    def __init__(self):
        self.catalog_name = "dev"
        self.silver_layer_schema_name = "swatch_staging"
        self.bronze_layer_schema_name = "swatch_raw"
        self.user_profile_table_name = "user_profile"
        self.workout_session__table_name = "workout_session"
        self.complete_workout_table_name = "completed_workouts"
        self.device_registration_table_name = 'device_registration'
        self.bpm_table = 'bpm'
        self.workout_bpm_table_name = "workout_bpm"
        self.gym_attendance_table = 'gym_attendance'

    

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# CDC Events -> Update , New , delete
# Ignore the delete events
class CDCUpsert:
    def __init__(self,merge_query,view_name):
        self.merge_query = merge_query
        self.view_name = view_name
    
    # ForEachBatch doesn't use the state store , Hence Spark doesn't persis any state.
    def upsert(self,micro_batch_df,batch_id):
        # Take the latest record within the microbatch
        window = Window.partitionBy("user_id").orderBy(col("event_timestamp").desc())
        micro_batch_latest_df = (micro_batch_df
                                 .withColumn("event_type",lower(col("event_type")))
                                 .where("event_type in ('update','new')")
                                 .withColumn("rk",row_number().over(window))
                                 .filter(col("rk") == 1)
                                )
        micro_batch_latest_df.createOrReplaceTempView(self.view_name)
        micro_batch_latest_df._jdf.sparkSession().sql(self.merge_query)


class UserProfileCDC(SilverLayer):
    def __init__(self):
        super().__init__()
    
    def load_raw_data(self):
        # read from the raw layer table
        raw_df = (spark
                  .readStream
                  .table(f"{self.catalog_name}.{self.bronze_layer_schema_name}.{self.user_profile_table_name}")
                  .selectExpr("user_id","update_type","timestamp","dob","sex","gender","first_name","last_name","address.*")
                  .withColumnRenamed("update_type","event_type")
                  .withColumnRenamed("timestamp","event_timestamp")
                  .withColumn("event_timestamp",col("event_timestamp").cast("timestamp"))
                  .withColumn("dob",to_date(col("dob"),"mm/dd/yyyy"))
                  )
        return raw_df
    
    merge_query = f""" 
         MERGE INTO dev.swatch_staging.user_profile as T
         using cdc_user_profile_view as S
         on T.user_id = S.user_id and T.event_timestamp > S.event_timestamp
         when matched then update set *
         when not matched then insert *
        """
    
    cdc = CDCUpsert(merge_query,"cdc_user_profile_view")

    def apply_cdc_changes(self,raw_df):
        streaming_query = (raw_df
                           .writeStream
                           .queryName("user_profile_cdc_streaming")
                           .format("delta")
                           .foreachBatch(self.cdc.upsert)
                           .outputMode("update")
                           .trigger(processingTime="10 seconds")
                           .option("checkpointLocation", "/Volumes/dev/swatch_staging/user_profile_checkpoint")
                           .start()
                           )
        return streaming_query

In [0]:
user_profile = UserProfileCDC()
streaming_query = user_profile.apply_cdc_changes(user_profile.load_raw_data())

In [0]:
streaming_query.stop()

In [0]:
%sql
select * from dev.swatch_staging.user_profile;

user_id,event_timestamp,dob,sex,gender,first_name,last_name,street_address,city,state,zip
11745,2023-03-10T12:34:41Z,1955-01-29,F,F,Shannon,Reyes,3105 Bowers Expressway,Long Beach,CA,90808
12140,2023-03-10T12:33:45Z,1999-01-02,M,M,Robert,Castillo,68994 Steven Vista,Pearblossom,CA,93553
12227,2023-03-10T12:40:28Z,1949-01-11,F,F,Courtney,Sheppard,47754 Angela Plaza Apt. 135,Los Angeles,CA,90010
12474,2023-03-10T12:26:08Z,1939-01-25,M,M,Matthew,Phillips,02648 Wilkins Cliffs Suite 998,San Fernando,CA,91340
13559,2023-03-10T12:34:10Z,1980-01-06,F,F,Victoria,Smith,634 Acevedo Mountain,Santa Monica,CA,90405
13937,2023-03-11T12:50:37Z,1982-01-26,M,M,Matthew,Johnson,9231 Edward Throughway Suite 072,Toluca Lake,CA,91610
14232,2023-03-11T12:51:38Z,1979-01-04,M,M,Edward,Smith,41444 Noble Cape Suite 390,North Hollywood,CA,91606
14508,2023-03-11T12:50:25Z,1936-01-28,M,M,Justin,Eaton,04952 Lori Plain,Sierra Madre,CA,91024
14633,2023-03-11T12:47:10Z,1997-01-04,F,F,Hannah,Fuller,81346 Obrien Streets,Gardena,CA,90249
15149,2023-03-11T12:51:42Z,1972-01-30,M,M,Cameron,Vasquez,95932 Gary Ridges,Los Angeles,CA,90018


In [0]:
from pyspark.sql.functions import *

class CompleteWorkoutIngestion(SilverLayer):
    def __init__(self):
        super().__init__()

    def compute_workout(self):
        # Stream to Stream Join 
        # Both of the stream will be save to state store to find the matching events.

        # Stream 1 - Start Events
        workout_start_df = (spark
                  .readStream
                  .table(f"{self.catalog_name}.{self.bronze_layer_schema_name}.{self.workout_session__table_name}")
                  .withColumnRenamed("timestamp","start_time")
                  .withColumn("start_time",col("start_time").cast("timestamp"))
                  .filter(col("action") == "start")
                  # Assumption - Workout can go max for 3 hours
                  # Events which is older than 180 minutes of the latest event will be dropped from state store.
                  # Latest Event - 10:20:15 AM , all the events before 07:20:15 AM will be dropped from state store.

                  # Workout start events at 10:20:15 AM
                  .withWatermark("start_time", "185 minutes")
                  )
        
         # Stream 1 - Stop Events
        workout_end_df = (spark
                  .readStream
                  .table(f"{self.catalog_name}.{self.bronze_layer_schema_name}.{self.workout_session__table_name}")
                  .withColumnRenamed("timestamp","end_time")
                  .withColumn("end_time",col("end_time").cast("timestamp"))
                  .filter(col("action") == "stop")
                  .withWatermark("end_time", "60 seconds")
                  )
        
        # Stream to Stream inner Join
        joined_condition = [workout_start_df.user_id == workout_end_df.user_id , workout_start_df.workout_id == workout_end_df.workout_id , workout_start_df.session_id == workout_end_df.session_id]
        
        joined_df = (workout_start_df
                     .join(workout_end_df,joined_condition,"inner")
                     .select(workout_start_df.user_id,workout_start_df.workout_id,workout_start_df.session_id,workout_start_df.start_time,workout_end_df.end_time)
                     )
        return joined_df

    
    def persist_complete_workout_df(self,joined_df):
        complete_workout_streaming_query = (joined_df
                        .writeStream
                        .queryName("complete_workout_streaming_query")
                        .format("delta")
                        .outputMode("append")
                        .trigger(processingTime="10 seconds")
                        .option("checkpointLocation", "/Volumes/dev/swatch_staging/completed_workouts_checkpoint")
                        .toTable(f"{self.catalog_name}.{self.silver_layer_schema_name}.{self.complete_workout_table_name}")
                        )
        return complete_workout_streaming_query



In [0]:
complete_workout = CompleteWorkoutIngestion()
df = complete_workout.compute_workout()
complete_workout_streaming_query=complete_workout.persist_complete_workout_df(df)

In [0]:
complete_workout_streaming_query.stop()

In [0]:
%sql
select * from dev.swatch_staging.completed_workouts;

user_id,workout_id,session_id,start_time,end_time
13559,1,2,2023-03-11T19:05:00Z,2023-03-11T19:55:00Z
15149,1,1,2023-03-12T08:35:00Z,2023-03-12T09:35:00Z
12474,1,1,2023-03-11T08:05:00Z,2023-03-11T09:10:00Z
14508,1,1,2023-03-12T08:05:00Z,2023-03-12T08:55:00Z
14633,1,2,2023-03-12T18:05:00Z,2023-03-12T18:35:00Z
14633,1,1,2023-03-12T08:05:00Z,2023-03-12T08:45:00Z
13559,1,1,2023-03-11T08:20:00Z,2023-03-11T09:25:00Z
12227,1,1,2023-03-11T08:45:00Z,2023-03-11T09:50:00Z
12140,1,1,2023-03-11T08:20:00Z,2023-03-11T08:55:00Z
14508,1,2,2023-03-12T18:15:00Z,2023-03-12T18:50:00Z


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

class WorkoutBPMIngestion(SilverLayer):
    def __init__(self):
        super().__init__()

    # Workout stream needs to be joined with the device_registration (static dataframe)
    def load_complete_workout_stream(self):
        device_registration_df = (spark
                                  .read
                                  .table(f"{self.catalog_name}.{self.bronze_layer_schema_name}.{self.device_registration_table_name}")
                                  )
        
        # Stream to static DF join doesn't store any state in the statestore
        workout_stream_df = (spark
                          .readStream
                          .table(f"{self.catalog_name}.{self.silver_layer_schema_name}.{self.complete_workout_table_name}")
                          .join(device_registration_df,"user_id","inner")
                          .select(device_registration_df.user_id,device_registration_df.device_id,col('workout_id'),col('session_id'),col('start_time'),col('end_time'))
                          .withWatermark("end_time", "60 seconds")
                          )
        return workout_stream_df
    
    def load_bpm_stream(self):
        bpm_stream_df = (spark
                      .readStream
                      .table(f"{self.catalog_name}.{self.bronze_layer_schema_name}.{self.bpm_table}")
                      .withColumnRenamed("time","event_time")
                      .withColumn("event_time",col("event_time").cast("timestamp"))
                      .withWatermark("event_time", "60 seconds")
                      )
        return bpm_stream_df
    
    # bpm_event_time must be between workout_start_time and workout_end_time
    # Workout can be of maximum 3 hours
    # BPM event can only be used for max 3 hours. Lets say BPM event send at start of workout , so can be used max for 3 hours window.
    def join_bpm_and_workout(self,bpm_stream_df,workout_stream_df):
        join_condition = [workout_stream_df.device_id == bpm_stream_df.device_id, bpm_stream_df.event_time >= workout_stream_df.start_time, bpm_stream_df.event_time <= workout_stream_df.end_time,workout_stream_df.end_time < bpm_stream_df.event_time + expr('interval 3 hour')]

        bpm_workout_stream_df = (workout_stream_df
                              .join(bpm_stream_df, join_condition, "inner")
                              .select(workout_stream_df.user_id,workout_stream_df.device_id,workout_stream_df.workout_id,workout_stream_df.session_id,workout_stream_df.start_time,workout_stream_df.end_time,bpm_stream_df.heartrate)
                              )
        
        return bpm_workout_stream_df
    
    def persist_bpm_workout_df(self,bpm_workout_stream_df):
        bpm_workout_stream_query = (bpm_workout_stream_df
                 .writeStream
                 .queryName("bpm_workout_stream_query")
                 .outputMode("append")
                 .trigger(processingTime="10 seconds")
                 .option("checkpointLocation", "/Volumes/dev/swatch_staging/workout_bpm_checkpoint")
                 .toTable(f"{self.catalog_name}.{self.silver_layer_schema_name}.{self.workout_bpm_table_name}")
                 )
        return bpm_workout_stream_query

    

In [0]:
workout_bpm = WorkoutBPMIngestion()
workout_stream = workout_bpm.load_complete_workout_stream()
bpm_stream = workout_bpm.load_bpm_stream()
bpm_workout_stream = workout_bpm.join_bpm_and_workout(bpm_stream,workout_stream)
bpm_workout_stream_query= workout_bpm.persist_bpm_workout_df(bpm_workout_stream)


In [0]:
bpm_workout_stream_query.stop()

In [0]:
%sql
select distinct user_id from dev.swatch_staging.workout_bpm;

user_id
14633
